# 190724第三次CNN建模

## Model: Inception_V3 under Keras
## Dropout: Y
## Data Augmentation: N

In [1]:
# 設定路徑
import os, shutil

train_dir = "C:/Users/1907075/Project/OM_train"
validation_dir = "C:/Users/1907075/Project/OM_validation"
test_dir = "C:/Users/1907075/Project/OM_test"
train_NG_dir = "C:/Users/1907075/Project/OM_train/1"
train_OK_dir = "C:/Users/1907075/Project/OM_train/2"
validation_NG_dir = "C:/Users/1907075/Project/OM_validation/1"
validation_OK_dir = "C:/Users/1907075/Project/OM_validation/2"
test_NG_dir = "C:/Users/1907075/Project/OM_test/1"
test_OK_dir = "C:/Users/1907075/Project/OM_test/2"

print("data counts:")
print("training_NG", len(os.listdir(train_NG_dir)))
print("training_OK", len(os.listdir(train_OK_dir)))
print("validation_NG", len(os.listdir(validation_NG_dir)))
print("validation_OK", len(os.listdir(validation_OK_dir)))
print("test_NG", len(os.listdir(test_NG_dir)))
print("test_OK", len(os.listdir(test_OK_dir)))

data counts:
training_NG 724
training_OK 824
validation_NG 300
validation_OK 300
test_NG 22
test_OK 23


### Build convolutional models from pretrained model given in Keras
#### We would like to apply Inception_V3 Model within this training model.
##### Keras Reference:
    https://keras.io/applications/
    https://keras.io/zh/applications/
    https://www.topbots.com/14-design-patterns-improve-convolutional-neural-network-cnn-architecture/

In [18]:
# import Inception_V3 model from Keras
from keras.applications import inception_v3

conv_base = InceptionV3(include_top=False, 
                         weights='imagenet', 
                         input_shape=(640,480,3))

conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 640, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 319, 239, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 319, 239, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 319, 239, 32) 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

## 在資料集上執行convolutional base 卷積基底並輸出到硬碟上
### 此方法不允許使用 DataAugmentation

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_dir = "C:/Users/1907075/Project/OM_train"
validation_dir = "C:/Users/1907075/Project/OM_validation"
test_dir = "C:/Users/1907075/Project/OM_test"

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 18, 13, 2048))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                           target_size=(640,480),
                                           batch_size=batch_size,
                                           class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch) # 從批次影像中萃取特徵
        features[i * batch_size : (i+1) * batch_size] = features_batch
        labels[i * batch_size : (i+1) * batch_size] = labels_batch
        i += 1
        print(i, end=' ')  # 檢視特徵萃取進度
        if i * batch_size >= sample_count:  # 當萃取樣本數超過設定的樣本數時，執行break中斷
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 1548) # Numbers of training samples
validation_features, validation_labels = extract_features(validation_dir, 600)
test_features, test_labels = extract_features(test_dir, 45)

In [ ]:
# 將資料攤平成convolutional layer最後一層的大小來提供給 dense 層 (sample num, convolutional layer dimensions)
train_features = np.reshape(train_features, (1548, 18*13*2048))
validation_features = np.reshape(validation_features, (600, 18*13*2048))
test_features = np.reshape(test_features, (45, 18*13*2048))

In [ ]:
# 建立 dense 層分類器
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=18*13*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
        
model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['acc'])
          
history = model.fit(train_features, train_labels,
                   epochs=30,
                   batch_size=20,
                   validation_data=(validation_features, validation_labels))

model.save('OM_Check_InceptionV3_1.h5')

In [ ]:
# 顯示訓練和驗證週期的損失值和準確度曲線

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()